In [21]:
from pyspark.sql.types import IntegerType
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
import copy
import sys

sys.path.append("..")  # Adjust path if needed
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.Processer import *
from dags.lib.pt_utils import *
from pyspark.sql.utils import AnalysisException
from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode

def create_spark_local_session():
    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of dags.lib.IncrementalLoader failed: Traceback (most recent call last):
  File "c:\Users\josub\.conda\envs\spark311\Lib\site-packages\IPython\extensions\autoreload.py", line 283, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\josub\.conda\envs\spark311\Lib\site-packages\IPython\extensions\autoreload.py", line 483, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "c:\Users\josub\.conda\envs\spark311\Lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\notebooks\..\dags\lib\IncrementalLoader.py", line 7, in <module>
    from lib.pt_utils import get_logger, gcs_path_exists
ModuleNotFoundError: No mod

In [13]:
spark = create_spark_local_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [ ]:
is_gcs_enabled= "False"
if is_gcs_enabled.lower() == 'true':
    is_gcs_enabled = True
else:
    is_gcs_enabled = False

spark, base_path = get_spark_and_path(is_gcs_enabled)

landing_path = '..\data\letstalk_landing_zone_bdma' #get_landing_path(base_path)
trusted_path ='..\data\letstalk_trusted_zone_bdma'

path= '..\data\letstalk_trusted_zone_bdma\control_table'

try:
    delta_table = DeltaTable.forPath(spark, path)
    print("Delta table exists.")
except AnalysisException:
    print("Delta table does not exist. Starting initialization")
    movies_subpath = r'delta_tmdb\database\movie'
    genre_subpath  = r'delta_tmdb\database\genre'
    movies_genre_subpath = r'delta_tmdb\database\movie_genre'

    loader = IncrementalLoader(spark, landing_path, movies_subpath)
    df = loader.get_new_data()
    loader_genre = IncrementalLoader(spark, landing_path, genre_subpath)
    df_genre= loader_genre.get_new_data()
    loader_mov_gen = IncrementalLoader(spark, landing_path, movies_genre_subpath)
    df_mov_gen= loader_mov_gen.get_new_data()

    processor = TMDBProcessor(spark, df)

    processor.ensure_schema()
    processor.normalize_text(['overview'])
    processor.remove_clear_duplicates()
    processor.remove_hidden_duplicates(['film_id'], ['ingestion_time'], True)

    processor.set_genre_df(df_genre)
    processor.ensure_schema_genres()

    processor.genre_df = processor.genre_df.withColumn("genre", lower(col("genre")))
    processor.genre_df = processor.genre_df.withColumn("genre", regexp_replace(col("genre"), r"http\S+|www\.\S+", " "))
    processor.genre_df = processor.genre_df.withColumn("genre", regexp_replace(col("genre"), r"[^a-zA-Z\s]", " "))
    processor.genre_df.dropDuplicates()
    window = Window.partitionBy(*["genre"]).orderBy(*["genre_id"])
    processor.genre_df= (
            processor.genre_df.withColumn("row_num", F.row_number().over(window)).filter(F.col("row_num") == 1).drop("row_num")
            )

    processor.set_movie_genre_df(df_mov_gen)
    processor.ensure_schema_movie_genres()
    processor.movie_genre_df.dropDuplicates()

    processor.static_dump(trusted_path)

INFO:dags.lib.pt_utils:False
INFO:dags.lib.pt_utils:False
..\data\letstalk_trusted_zone_bdma\control_table
Delta table exists.


In [16]:
#static dataset to trusted  - one time operation
absolute_path_to_landing = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_landing_zone_bdma'
absolute_path_to_trusted = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_trusted_zone_bdma'
movies_subpath = r'delta_tmdb\database\movie'
genre_subpath  = r'delta_tmdb\database\genre'
movies_genre_subpath = r'delta_tmdb\database\movie_genre'

loader = IncrementalLoader(spark, absolute_path_to_landing, movies_subpath)
df = loader.get_new_data()
loader_genre = IncrementalLoader(spark, absolute_path_to_landing, genre_subpath)
df_genre= loader_genre.get_new_data()
loader_mov_gen = IncrementalLoader(spark, absolute_path_to_landing, movies_genre_subpath)
df_mov_gen= loader_mov_gen.get_new_data()

processor = TMDBProcessor(spark, df)

processor.ensure_schema()
processor.normalize_text(['overview'])
processor.remove_clear_duplicates()
processor.remove_hidden_duplicates(['film_id'], ['ingestion_time'], True)

processor.set_genre_df(df_genre)
processor.ensure_schema_genres()

processor.genre_df = processor.genre_df.withColumn("genre", lower(col("genre")))
processor.genre_df = processor.genre_df.withColumn("genre", regexp_replace(col("genre"), r"http\S+|www\.\S+", " "))
processor.genre_df = processor.genre_df.withColumn("genre", regexp_replace(col("genre"), r"[^a-zA-Z\s]", " "))
processor.genre_df.dropDuplicates()
window = Window.partitionBy(*["genre"]).orderBy(*["genre_id"])
processor.genre_df= (
         processor.genre_df.withColumn("row_num", F.row_number().over(window)).filter(F.col("row_num") == 1).drop("row_num")
        )

processor.set_movie_genre_df(df_mov_gen)
processor.ensure_schema_movie_genres()
processor.movie_genre_df.dropDuplicates()

processor.static_dump(absolute_path_to_trusted)

print(processor.df)
print(processor.genre_df)
print(processor.movie_genre_df)


CDF not available — doing full load
CDF not available — doing full load
CDF not available — doing full load


INFO:root:Removed 0 simple duplicate(s)
INFO:root:Removed 0 hidden duplicate(s)


+-------+--------------------+---------------------------+-----------------+--------------------+------------+----------+---------+-------+-----+----------+------------+----------+--------------------+
|film_id|               title|             original_title|original_language|            overview|release_date|   revenue|   budget|runtime|adult|popularity|vote_average|vote_count|      ingestion_time|
+-------+--------------------+---------------------------+-----------------+--------------------+------------+----------+---------+-------+-----+----------+------------+----------+--------------------+
|     19|          Metropolis|                 Metropolis|               de|in a futuristic c...|  1927-03-06|    650422|  1300000|  149.0|false|    17.498|         8.2|      1598|2025-05-14 08:35:...|
|     22|Pirates of the Ca...|       Pirates of the Ca...|               en|jack sparrow  a f...|  2003-07-09| 655011224|140000000|  143.0|false|    56.942|         7.7|     15060|2025-05-14 0

In [ ]:
#api dataset to trusted  - to be repeated
absolute_path_to_landing = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_landing_zone_bdma'
table_subpaths = [r'delta_tmdb\now_playing',r'delta_tmdb\trending',r'delta_tmdb\upcoming']
absolute_path_to_trusted = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_trusted_zone_bdma'

for i in range(len(table_subpaths)):
    loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpaths[i])
    df = loader.get_new_data()
    if i==0:
        processor= TMDBProcessor(spark, df)

        processor.ensure_schema()
        processor.normalize_text(['overview'])
        processor.ensure_schema_movie_genres()
    
    else:
        processor1= TMDBProcessor(spark, df)

        processor1.ensure_schema()
        processor1.normalize_text(['overview'])
        processor1.ensure_schema_movie_genres()
        processor.combine_dfs(processor1)
    
processor.merge_with_trusted(absolute_path_to_trusted)


CDF not available — doing full load
CDF not available — doing full load


INFO:root:Processor -  df:100 records | movie_genre:270
INFO:root:Processor -  df:140 records | movie_genre:377
INFO:root:Merged -  df:240 records | movie_genre:647


CDF not available — doing full load


INFO:root:Processor -  df:240 records | movie_genre:647
INFO:root:Processor -  df:100 records | movie_genre:264
INFO:root:Merged -  df:340 records | movie_genre:911
INFO:root:Removed 0 simple duplicate(s)
INFO:root:Removed 264 hidden duplicate(s)


In [31]:
absolute_path_to_trusted = r'C:\Users\josub\Desktop\BDMA\Barcelona\Big_Data_Management\SmallProject\LetsTalk\data\letstalk_trusted_zone_bdma'
table_subpath = r'\movie'
df = spark.read.format("delta").load(absolute_path_to_trusted+table_subpath)

print(df.show(10))

+-------+--------------------+--------------------+-----------------+--------------------+------------+---------+--------+-------+-----+----------+------------+----------+--------------------+
|film_id|               title|      original_title|original_language|            overview|release_date|  revenue|  budget|runtime|adult|popularity|vote_average|vote_count|      ingestion_time|
+-------+--------------------+--------------------+-----------------+--------------------+------------+---------+--------+-------+-----+----------+------------+----------+--------------------+
|      5|          Four Rooms|          Four Rooms|               en|it s ted the bell...|  1995-12-09|  4257354| 4000000|   98.0|false|    16.877|         5.8|      1753|2025-05-14 08:34:...|
|     17|            The Dark|            The Dark|               en|ad le and her dau...|  2005-09-28|        0|       0|   87.0|false|     8.127|         5.9|       177|2025-05-14 08:34:...|
|     27|             9 Songs|     